In [25]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import time
from PIL import Image
from IPython.display import display, Image as IPImage
from dataclasses import dataclass
from typing import Tuple, List, Any
import math

In [26]:
@dataclass
class SceneConfig:
    """Конфигурация сцены"""
    grid_spacing: float = 4.0
    sphere_radius_range: Tuple[float, float] = (0.8, 1.2)
    sphere_y_range: Tuple[float, float] = (-2.0, 2.0)
    sphere_color_saturation: Tuple[float, float] = (0.8, 1.0)
    sphere_color_value: Tuple[float, float] = (0.8, 1.0)
    sphere_shininess: Tuple[float, float] = (30.0, 100.0)
    
    light_radius_range: Tuple[float, float] = (10.0, 20.0)
    light_phi_range: Tuple[float, float] = (0.0, math.pi/3)
    light_y_offset: float = 5.0
    light_hue_range: Tuple[float, float] = (0.0, 0.1)
    light_saturation_range: Tuple[float, float] = (0.0, 0.2)
    light_intensity_range: Tuple[float, float] = (1.2, 2.0)

In [27]:
def hsv_to_rgb(h: float, s: float, v: float) -> Tuple[float, float, float]:
    """Преобразование HSV в RGB"""
    h = h * 6.0
    hi = int(h)
    f = h - hi
    p = v * (1.0 - s)
    q = v * (1.0 - f * s)
    t = v * (1.0 - (1.0 - f) * s)
    
    if hi == 0: return (v, t, p)
    elif hi == 1: return (q, v, p)
    elif hi == 2: return (p, v, t)
    elif hi == 3: return (p, q, v)
    elif hi == 4: return (t, p, v)
    else: return (v, p, q)

In [28]:
class RayTracer:
    def __init__(self, width: int, height: int, num_spheres: int, num_lights: int, config: SceneConfig = None):
        self.width = width
        self.height = height
        self.num_spheres = num_spheres
        self.num_lights = num_lights
        self.config = config or SceneConfig()
        
        # Инициализация CUDA
        self._init_cuda()
        
        # Создание сцены
        self._create_scene()
        
        # Выделение памяти на GPU
        self._allocate_gpu_memory()
    
    def _init_cuda(self) -> None:
        """Инициализация CUDA kernel"""
        with open('kernel.cu', 'r') as f:
            kernel_code = f.read()
        self.mod = SourceModule(kernel_code)
        self.render = self.mod.get_function("render")
    
    def _create_spheres(self) -> None:
        """Создание сфер со случайными параметрами"""
        self.spheres = np.zeros(self.num_spheres, dtype=[
            ('center', np.float32, 3),
            ('radius', np.float32),
            ('color', np.float32, 3),
            ('shininess', np.float32)
        ])
        
        # Создаем сетку позиций
        positions = self._generate_grid_positions()
        np.random.shuffle(positions)
        
        # Инициализируем сферы
        for i in range(self.num_spheres):
            self.spheres[i]['center'] = np.array(positions[i], dtype=np.float32)
            self.spheres[i]['radius'] = np.random.uniform(*self.config.sphere_radius_range)
            
            # Генерация цвета
            hue = np.random.uniform(0, 1)
            saturation = np.random.uniform(*self.config.sphere_color_saturation)
            value = np.random.uniform(*self.config.sphere_color_value)
            
            rgb = hsv_to_rgb(hue, saturation, value)
            self.spheres[i]['color'] = np.array(rgb, dtype=np.float32)
            self.spheres[i]['shininess'] = np.random.uniform(*self.config.sphere_shininess)
    
    def _generate_grid_positions(self) -> List[List[float]]:
        """Генерация позиций в сетке"""
        grid_size = int(np.ceil(np.sqrt(self.num_spheres)))
        positions = []
        
        for i in range(grid_size):
            for j in range(grid_size):
                x = (i - grid_size/2) * self.config.grid_spacing + np.random.uniform(-1, 1)
                z = (j - grid_size/2) * self.config.grid_spacing - 10 + np.random.uniform(-1, 1)
                y = np.random.uniform(*self.config.sphere_y_range)
                positions.append([x, y, z])
        
        return positions
    
    def _create_lights(self) -> None:
        """Создание источников света со случайными параметрами"""
        self.lights = np.zeros(self.num_lights, dtype=[
            ('position', np.float32, 3),
            ('color', np.float32, 3),
            ('intensity', np.float32)
        ])
        
        for i in range(self.num_lights):
            # Позиция в сферических координатах
            theta = np.random.uniform(0, 2 * np.pi)
            phi = np.random.uniform(*self.config.light_phi_range)
            radius = np.random.uniform(*self.config.light_radius_range)
            
            # Преобразование в декартовы координаты
            x = radius * np.sin(phi) * np.cos(theta)
            y = radius * np.cos(phi) + self.config.light_y_offset
            z = radius * np.sin(phi) * np.sin(theta)
            
            self.lights[i]['position'] = np.array([x, y, z], dtype=np.float32)
            
            # Цвет света
            hue = np.random.uniform(*self.config.light_hue_range)
            saturation = np.random.uniform(*self.config.light_saturation_range)
            rgb = hsv_to_rgb(hue, saturation, 1.0)
            
            self.lights[i]['color'] = np.array(rgb, dtype=np.float32)
            self.lights[i]['intensity'] = np.random.uniform(*self.config.light_intensity_range)
    
    def _create_scene(self) -> None:
        """Создание всей сцены"""
        self._create_spheres()
        self._create_lights()
    
    def _allocate_gpu_memory(self) -> None:
        """Выделение памяти на GPU и копирование данных"""
        self.d_spheres = cuda.mem_alloc(self.spheres.nbytes)
        self.d_lights = cuda.mem_alloc(self.lights.nbytes)
        self.d_output = cuda.mem_alloc(self.width * self.height * 4)
        
        cuda.memcpy_htod(self.d_spheres, self.spheres)
        cuda.memcpy_htod(self.d_lights, self.lights)
    
    def render_scene(self) -> Tuple[np.ndarray, float]:
        """Рендеринг сцены"""
        block_size = (16, 16, 1)
        grid_size = (
            (self.width + block_size[0] - 1) // block_size[0],
            (self.height + block_size[1] - 1) // block_size[1]
        )
        
        start_time = time.time()
        self.render(
            self.d_output,
            np.int32(self.width),
            np.int32(self.height),
            self.d_spheres,
            np.int32(self.num_spheres),
            self.d_lights,
            np.int32(self.num_lights),
            block=block_size,
            grid=grid_size
        )
        cuda.Context.synchronize()
        render_time = time.time() - start_time
        
        output = np.empty((self.height, self.width, 4), dtype=np.uint8)
        cuda.memcpy_dtoh(output, self.d_output)
        
        return output, render_time
    
    def save_image(self, output: np.ndarray, filename: str) -> Image.Image:
        """Сохранение результата в файл"""
        img = Image.fromarray(output, 'RGBA')
        img.save(filename)
        return img
    
    def __del__(self):
        """Освобождение ресурсов GPU"""
        if hasattr(self, 'd_spheres'):
            self.d_spheres.free()
        if hasattr(self, 'd_lights'):
            self.d_lights.free()
        if hasattr(self, 'd_output'):
            self.d_output.free()

In [31]:
# Параметры рендеринга
width = 1920
height = 1080
num_spheres = 10
num_lights = 2

# Создание ray tracer с настраиваемой конфигурацией
config = SceneConfig(
    grid_spacing=4.0,
    sphere_radius_range=(0.8, 1.2),
    light_radius_range=(10.0, 20.0),
    light_intensity_range=(1.2, 2.0)
)

tracer = RayTracer(width, height, num_spheres, num_lights, config)

# Рендеринг сцены
output, render_time = tracer.render_scene()

# Сохранение и отображение результата
img = tracer.save_image(output, 'output.bmp')
print(f"Rendering completed in {render_time:.2f} seconds")

Rendering completed in 0.01 seconds
